## Generating solar scenarios using PGscen ##

In this notebook we will use PGscen to create a scenarios for photovoltaic generator output across the Texas 7k system. The output generated here can also be created using the ```pgscen-solar``` command line tool.

As in the case of the load and wind asset scenarios, we begin by creating the scenario generation parameters and setting up the input datasets.

In [ ]:
from pathlib import Path
import pandas as pd
from pgscen.command_line import (
    load_solar_data, split_actuals_hist_future, split_forecasts_hist_future)

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.dates as mdates
%matplotlib inline
plt.rcParams['figure.figsize'] = [19, 11]
from IPython.display import display

start_date = '2018-03-03'
cur_path = Path("day-ahead_solar.ipynb").parent.resolve()
data_dir = Path(cur_path, '..', "data").resolve()

scenario_count = 1000
scen_start_time = pd.to_datetime(' '.join([start_date, "06:00:00"]), utc=True)
scen_timesteps = pd.date_range(start=scen_start_time, periods=24, freq='H')

(solar_site_actual_df, solar_site_forecast_df,
     solar_meta_df) = load_solar_data()

(solar_site_actual_hists,
     solar_site_actual_futures) = split_actuals_hist_future(
            solar_site_actual_df, scen_timesteps)

(solar_site_forecast_hists,
     solar_site_forecast_futures) = split_forecasts_hist_future(
            solar_site_forecast_df, scen_timesteps)

fig, (hist_ax, future_ax) = plt.subplots(figsize=(17, 6), nrows=1, ncols=2)
title_args = dict(weight='semibold', size=19)
actual_clr, fcst_clr = "#430093", "#D9C800"
plt_asset = 'Angelo Solar'

hist_ax.set_title("History", **title_args)
hist_ax.plot(solar_site_actual_hists[plt_asset][-250:], c=actual_clr)
hist_ax.plot(solar_site_forecast_hists['Forecast_time'][-250:],
             solar_site_forecast_hists[plt_asset][-250:],
             c=fcst_clr)
hist_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

future_ax.set_title("Future", **title_args)
future_ax.plot(solar_site_actual_futures[plt_asset][:250], c=actual_clr)
future_ax.plot(solar_site_forecast_futures['Forecast_time'][:250],
               solar_site_forecast_futures[plt_asset][:250], c=fcst_clr)
future_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

lgnd_ptchs = [Patch(color=actual_clr, alpha=0.53, label="Actuals"),
              Patch(color=fcst_clr, alpha=0.53, label="Forecasts")]

_ = fig.legend(handles=lgnd_ptchs, frameon=False, fontsize=17, ncol=2, loc=8,
               bbox_to_anchor=(0.5, -0.04), handletextpad=0.7)

Like the wind scenario engine, the solar engine creates a unique regularization penalty for each pair of generators in the GEMINI model based on the physical distance between them, which is calculated using generator metadata. Unlike the wind engine, this is handled internally in the solar engine.

In [ ]:
from pgscen.engine import SolarGeminiEngine
import geopandas as gpd
from shapely.geometry import Point


se = SolarGeminiEngine(solar_site_actual_hists, solar_site_forecast_hists,
                       scen_start_time, solar_meta_df)

gdf = gpd.GeoDataFrame(
    se.meta_df, geometry=[Point(xy) for xy in zip(se.meta_df['longitude'],
                                                  se.meta_df['latitude'])]
    )

tx = gpd.read_file(Path(data_dir, "MetaData", "Texas_State_Boundary-shp.zip"))
ax = tx.plot(facecolor="none", edgecolor='black', linewidth=3, figsize=(18, 18))
gdf.plot(ax=ax, marker='o', linewidth=0, markersize=143, c='green')
plt.title('ERCOT Solar Sites', fontsize=50)
_ = ax.axis('off')

se.get_solar_reg_param().round(4)
se.fit_solar_model()

Solar scenario models are unique in that they consist of a series of linked models for different parts of the day. This includes a model for generator behavior during the daytime, as well as a series of conditional models describing dawn and dusk times.

Each of these models consists of a covariance matrix across all pairs of generators, and another covariance matrix across all pairs of day times included in the model.

In [ ]:
import numpy as np
from itertools import combinations as combn
from shapely.geometry import LineString

fig, axarr = plt.subplots(figsize=(19, 19), nrows=2, ncols=2)
use_conds = min(se.cond_count, 3)

for ax, mdl_lbl in zip(axarr.flatten(),
                       ['day'] + [('cond', i) for i in range(use_conds)]):
    plt_info = se.gemini_dict[mdl_lbl]

    asset_cov = plt_info['gemini_model'].asset_cov
    D = np.sqrt(np.diag(asset_cov))
    corr_mat = (D ** -1) * asset_cov * (D ** -1)

    cnct_dict = {
        (asset1, asset2): {
            'geometry': LineString([gdf.geometry[asset1], gdf.geometry[asset2]]),
            'corr_val': corr_mat.loc[asset1, asset2]
            }
        for asset1, asset2 in combn(asset_cov.index, 2)
        if corr_mat.loc[asset1, asset2] >= 0.4
        }

    tx.plot(ax=ax, facecolor="none", edgecolor='black', linewidth=2.3)
    gdf.loc[plt_info['asset_list']].plot(ax=ax, marker='o', linewidth=0, markersize=71, c='green')

    edges_gdf = gpd.GeoDataFrame(cnct_dict).transpose()
    edges_gdf.plot(ax=ax, linewidth=(edges_gdf.corr_val - 0.4) ** (1 / 7),
                   alpha=0.31)

    start_hour = plt_info['scen_start_time'] - pd.Timedelta(hours=6)
    end_hour = start_hour + pd.Timedelta(hours=plt_info['num_of_horizons'] - 1)
    
    ax.text(0.5, 1.03,
            "{} generators\n{} to {}".format(len(plt_info['asset_list']),
                                             start_hour.strftime("%H:%M"),
                                             end_hour.strftime("%H:%M")),
            size=27, ha='center', va='bottom', transform=ax.transAxes)

for ax in axarr.flatten():
    _ = ax.axis('off')
    
fig.tight_layout()

Like wind, scenario generation can also take a few minutes of runtime.

In [ ]:
se.create_solar_scenario(scenario_count, solar_site_forecast_futures)

In [ ]:
from ipywidgets import interact, Dropdown
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


def plot_solar_scenarios(solar_site):
    ax = tx.plot(facecolor="none", edgecolor='black', figsize=(18, 18), linewidth=2.3)
    gdf.plot(ax=ax, marker='o', linewidth=0, markersize=71, c='green')

    cntr_x, cntr_y = se.meta_df.longitude.mean(), se.meta_df.latitude.mean()
    site_x, site_y = se.meta_df.loc[solar_site, ['longitude', 'latitude']]
    xdiff, ydiff = site_x - cntr_x, site_y - cntr_y

    cntr_dist = (xdiff ** 2 + ydiff ** 2) ** 0.5
    site_cos, site_sin = xdiff / cntr_dist, ydiff / cntr_dist
    scen_x, scen_y = cntr_x + 8.5 * site_cos, cntr_y + 8.5 * site_sin

    scen_size = 7.5, 4
    scen_bbox = (scen_x - scen_size[0] / 2, scen_y - scen_size[1] / 2,
                 scen_size[0], scen_size[1])

    scen_ax = inset_axes(ax, width='100%', height='100%',
                         bbox_to_anchor=scen_bbox, bbox_transform=ax.transData,
                         loc=10, borderpad=0)
    
    ax.plot([site_x + 0 * site_cos, scen_x - 3.8 * site_cos],
            [site_y + 0 * site_sin, scen_y - 2 * site_sin],
            linewidth=2, c='red', alpha=0.53)
    
    for i in range(scenario_count):
        scen_ax.plot(se.scenarios['solar'].iloc[i][solar_site],
                     c='black', alpha=0.13, lw=0.2)

    plt_fcst = se.forecasts['solar'][solar_site]
    scen_ax.plot(plt_fcst, c=fcst_clr, alpha=0.47, lw=4.1)
    scen_ax.plot(solar_site_actual_futures.loc[plt_fcst.index, solar_site],
                 c=actual_clr, alpha=0.47, lw=4.1)

    quant_df = se.scenarios['solar'][solar_site].quantile([0.25, 0.75])
    scen_ax.fill_between(quant_df.columns, quant_df.iloc[0], quant_df.iloc[1],
                         color='red', alpha=0.31)

    lgnd_ptchs = [Patch(color='black', alpha=0.23, label="Scenarios"),
                  Patch(color='red', alpha=0.41, label="Interquartile Range"),
                  Patch(color=fcst_clr, alpha=0.81, label="Forecast"),
                  Patch(color=actual_clr, alpha=0.81, label="Actual")]

    _ = scen_ax.legend(handles=lgnd_ptchs, frameon=False,
                       fontsize=11, ncol=2, handletextpad=0.5)

    ax.axis('off')

    
w = Dropdown(options=se.asset_list, description="Scenarios for generator:",
             layout={'align_self': 'center'}, style={'description_width': 'initial'},
             disabled=False)
_ = interact(plot_solar_scenarios, solar_site=w)